# From Landsat 8 Scene to AOI Stacked Array 

This script will crop a previously downloaded Level-2 (on-demand) Landsat 8 scene to an area of interest (AOI), resulting in a stacked array.

**Note**: When running this code as is, it will take a long time and a lot of space to download the full Landsat scene from figshare (427.32MB zipped, and unzips at download).

In [3]:
# Import Packages
import os
from glob import glob

import geopandas as gpd
#from shapely.geometry import box
import numpy as np
from matplotlib import pyplot as plt, cm
from matplotlib.colors import ListedColormap
import rasterio as rio
from rasterio.mask import mask
from rasterio.plot import plotting_extent

import earthpy as et
import earthpy.spatial as es
import earthpy.plot as ep

#######################NOT WORKING: INSISTING IT IS A .PY FILES###############
# Make ONAQ site info retrievable
%run ../modules/retrieve-neon-site-boundary.ipynb

In [4]:
# # Get data and set working and outputs directory
# url='https://ndownloader.figshare.com/files/23372924'
# et.data.get_data(url=url, replace=True)

os.chdir(os.path.join(et.io.HOME, 'earth-analytics'))

output_subdir = os.path.join('data', 'Landsat')
if os.path.isdir(output_subdir) == False:
    os.mkdir(output_subdir)

output_dir = os.path.join(output_subdir, "outputs")
if os.path.isdir(output_dir) == False:
    os.mkdir(output_dir)

## Retrieve AOI shapefile

In [6]:
#############################AUTOMATION NOT ACCESSING DATA GRABBER#######################
# Download aoi extent from NEON
# url = 'https://www.neonscience.org/neon-terrestrial-field-site-boundaries-shapefile'
# et.data.get_data(url=url, replace=True)

# Create path to shapefile
terrestrial_sites_path = os.path.join(
    'data', 'earthpy-downloads',
    'Field_Sampling_Boundaries_2020',
    'terrestrialSamplingBoundaries.shp')

# # Read in extent as pandas geodataframe
# site_bounds = NEON_site_extent(
#     path_to_NEON_boundaries=terrestrial_sites_path,
#     site='ONAQ')

# Retrieving the boundaries of ONAQ site
NEON_boundaries = gpd.read_file(terrestrial_sites_path)
boundaries_indexed = NEON_boundaries.set_index(['siteID'])

site_bounds = boundaries_indexed.loc[['ONAQ']]
site_bounds.reset_index(inplace=True)

site_bounds

,siteID,domainNumb,domainName,siteType,siteName,siteHost,areaKm2,acres,geometry
0,ONAQ,D15,Great Basin,Core Terrestrial,Onaqui,Bureau of Land Management,67.774086,16747.341399,"POLYGON ((-112.42605 40.22358, -112.42129 40.2..."


## Explore Landsat 8 scene

In [ ]:
# Create bands path
l8_onaq_path = glob(os.path.join("data",
                    "earthpy-downloads", 
                    "LC080380322019061001T1-SC20200623214804", 
                    "*band*.tif"))

# Sort bands in numeric order
l8_onaq_path.sort()

# Import, stack onaq rasters 
l8_onaq, onaq_meta = es.stack(l8_onaq_path, nodata=-9999.0)

# Create landsat extent
landsat_extent = plotting_extent(l8_onaq[0], 
                                 onaq_meta["transform"])

In [ ]:
# # Visualize band data without -9999
ep.hist(l8_onaq, 
        title=["Band 1", "Band 2", "Band 3",
               "Band 4", "Band 5", "Band 6", 
               "Band 7"])

## Bring AOI and Landsat 8 scene together

In [ ]:
# Assure crs of aoi and raster are same 
print('site bounds crs: ', site_bounds.crs)
print('landsat bounds crs: 32612')
######################NOT SURE WHY NOT WORKING####################
# print('landsat crs: ', onaq_meta.crs)

# Reprojection of AOI shapefile
site_bounds_reproj = site_bounds.to_crs(32612)
print('site bounds reprojected crs: ', site_bounds_reproj.crs)

In [ ]:
# Crop stack extent of each band
crop_images = es.crop_all(l8_onaq_path, raster_out_path, site_bounds_reproj, overwrite=True)

stacked_path = os.path.join(output_dir, 'stacked_aoi.tif')

stacked_aoi, stacked_meta = es.stack(crop_images, stacked_path)

In [ ]:
# Plot new raster AOI extent
ep.plot_bands(stacked_aoi)